## Visdom可视化

### 安装

!pip install visdom

!python -m visdom.server

如果报错找不到路径，解决方式：在该目录下自己创建该文件夹

安装方式2：避免download问题

1. pip uninstall visdom
1. 从github下载visdom文件
2. cd 该目录下，pip install -e .
3. 在项目路径下 python -m visdom.server成功启动

### 使用

https://blog.csdn.net/weixin_41010198/article/details/117853358

In [2]:
from visdom import Visdom
import torch

loss = torch.rand(10)

global_step = torch.arange(10)


viz = Visdom()


viz.line([0.],[0.],win='train_loss',opts = dict(title='train_loss'))

for idx,lossi in enumerate(loss):
   
    viz.line([lossi.item()],[idx],win = 'train_loss',update='append')

Setting up a new session...


In [ ]:
!python -m visdom.server

In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

from visdom import Visdom

batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

#一般采用以下的写法，nn.Linear自带了自己的初始化方式，一般够用，不够的话就要自己写
#注意 nn.ReLU与 F.relu的区别

class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 10),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

net = MLP()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss()

viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))

global_step = 0

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)

        logits = net(data) #注意这里是logits
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')


        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Setting up a new session...


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.313021
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.197366
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.870139

Test set: Average loss: 0.0073, Accuracy: 5911/10000 (59%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.461465
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.340940
Train Epoch: 1 [40000/60000 (67%)]	Loss: 1.219751

Test set: Average loss: 0.0055, Accuracy: 6535/10000 (65%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.057201
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.079342
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.201055

Test set: Average loss: 0.0051, Accuracy: 6946/10000 (69%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.084290
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.983549
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.941677

Test set: Average loss: 0.0049, Accuracy: 7116/10000 (71%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.005059
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.869344
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.993601

Test set: Average loss: 0.0048

In [3]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

from visdom import Visdom

batch_size=200
learning_rate=0.01
epochs=2

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)



class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))
global_step = 0

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.to(device)

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Setting up a new session...


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.304898
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.298291
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.269844

Test set: Average loss: 0.0112, Accuracy: 2785.0/10000 (28%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.248003
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.227754
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.136341

Test set: Average loss: 0.0102, Accuracy: 3263.0/10000 (33%)

